In [1]:
# ============================================
# Pozo infinito 1D — PINN vs Diferencias Finitas (DF)
# Validación con E_n = (n*pi)^2, comparación y caché de resultados.
# TensorFlow 2.16+ / Keras 3
# ============================================

import os, math, csv, random, json
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import layers, ops as K

# ============ RUTA BASE ============
BASE_DIR = "/home/david/Schr-dingerPINNsUQValidation/PrimeraFase/ValidacionPozoInfinito/DiferenciasFinitas/ResultadosD"
os.makedirs(BASE_DIR, exist_ok=True)

def path(*args, is_dir=False):
    """Une rutas relativas dentro de BASE_DIR y crea el directorio padre."""
    p = os.path.join(BASE_DIR, *args)
    d = p if is_dir else os.path.dirname(p)
    os.makedirs(d, exist_ok=True)
    return p

# ============ SEMILLAS Y GPU ============
tf.keras.utils.set_random_seed(0)
np.random.seed(0)
random.seed(0)

# Memory growth
for g in tf.config.list_physical_devices("GPU"):
    try:
        tf.config.experimental.set_memory_growth(g, True)
    except Exception:
        pass

# (Opcional) Mixed precision para Tensor Cores
USE_MIXED_PRECISION = True
if USE_MIXED_PRECISION:
    keras.mixed_precision.set_global_policy("mixed_float16")
print("Política actual:", keras.mixed_precision.global_policy())

# ============ UTILIDADES ============
def trapz_np(y, x):
    y = y.astype(np.float32, copy=False)
    return float(np.trapezoid(y, x=x))

def l2_rel(y_true, y_pred):
    num = np.linalg.norm(y_true - y_pred)
    den = np.linalg.norm(y_true) + 1e-12
    return float(num / den)

def normalize_to_one(xs, psi):
    """Normaliza psi para que ∫|psi|^2 dx = 1 en [min(xs), max(xs)]."""
    val = trapz_np(np.abs(psi)**2, xs)
    if val <= 0:
        return psi, 0.0
    psi2 = psi / np.sqrt(val)
    return psi2, 1.0

# ============ ANALÍTICA ============
def analytic_psi(n, xs):
    return np.sqrt(2.0) * np.sin(n * math.pi * xs)

def analytic_E(n):
    return (n * math.pi)**2

# ============ DIFERENCIAS FINITAS ============
def fd_solve(n_modes=6, N=800):
    """
    Resuelve -psi'' = E psi en (0,1), Dirichlet psi(0)=psi(1)=0,
    con m = N-2 puntos interiores, h = 1/(N-1).
    Devuelve:
      xs_fd: m+2 puntos (incluye 0 y 1), psi_fd[k], E_fd[k] para k=1..n_modes
    """
    # Malla con extremos incluidos (0..1). Usamos interior para el sistema.
    xs = np.linspace(0.0, 1.0, N, dtype=np.float64)
    h = xs[1] - xs[0]
    m = N - 2  # puntos interiores
    # Matriz tridiagonal del -d2/dx2 con Dirichlet (interior)
    main = (2.0 / h**2) * np.ones(m)
    off  = (-1.0 / h**2) * np.ones(m-1)
    # Construcción densa (N pequeño). Para N grandes, usar scipy.sparse.
    A = np.diag(main) + np.diag(off, k=1) + np.diag(off, k=-1)
    # Autovalores y autovectores
    w, V = np.linalg.eigh(A)  # w ascendente
    # Seleccionamos los primeros n_modes
    nm = min(n_modes, m)
    E_fd = w[:nm].astype(float)  # ~ (n*pi)^2
    psi_fd = []
    for k in range(nm):
        v = V[:, k]
        # Insertar ceros en extremos para cumplir Dirichlet
        psi_full = np.zeros(N, dtype=float)
        psi_full[1:-1] = v
        # Normalizar a ∫|psi|^2 dx = 1
        psi_full, _ = normalize_to_one(xs, psi_full)
        # Alineación de signo (consistencia con sin positivo cerca de inicio)
        sgn = np.sign(np.dot(psi_full, analytic_psi(k+1, xs)))
        if sgn == 0: sgn = 1.0
        psi_full *= sgn
        psi_fd.append(psi_full.astype(float))
    return xs.astype(float), psi_fd, E_fd

# ============ PINN ============
class Sine(layers.Layer):
    def call(self, x):
        return K.sin(x)

def trig_nodal_factor(x, n: int):
    eps = K.cast(1e-12, x.dtype)
    s1  = K.sin(math.pi * x)
    sn  = K.sin(n * math.pi * x)
    ratio = sn / (s1 + eps)
    n_cast = K.cast(n, x.dtype)
    return K.where(K.abs(s1) < 1e-6, n_cast, ratio)

def make_net(n=1, hidden=64, use_sine=True):
    x_in = keras.Input(shape=(1,), dtype="float32")
    if use_sine:
        z = layers.Dense(hidden, activation=None,
                         kernel_initializer="glorot_uniform",
                         bias_initializer="zeros")(x_in)
        z = Sine()(z)
        z = layers.Dense(hidden, activation=None,
                         kernel_initializer="glorot_uniform",
                         bias_initializer="zeros")(z)
        z = Sine()(z)
    else:
        z = layers.Dense(hidden, activation="tanh",
                         kernel_initializer="glorot_uniform",
                         bias_initializer="zeros")(x_in)
        z = layers.Dense(hidden, activation="tanh",
                         kernel_initializer="glorot_uniform",
                         bias_initializer="zeros")(z)
    out = layers.Dense(1, activation=None,
                       kernel_initializer="glorot_uniform",
                       bias_initializer="zeros",
                       dtype="float32")(z)
    F = trig_nodal_factor(x_in, n)
    psi = K.multiply(x_in, (1.0 - x_in))
    psi = K.multiply(psi, F)
    psi = K.multiply(psi, out)
    return keras.Model(inputs=x_in, outputs=psi)

def second_derivative(model, x):
    x = tf.convert_to_tensor(x)
    x = tf.reshape(x, (-1,1))
    with tf.GradientTape(persistent=True) as t2:
        t2.watch(x)
        with tf.GradientTape() as t1:
            t1.watch(x)
            psi = model(x)
        psi_x = t1.gradient(psi, x)
    psi_xx = t2.gradient(psi_x, x)
    del t2
    return psi, psi_xx

@tf.function
def compute_losses(net, x_batch, E, lam):
    psi, psi_xx = second_derivative(net, x_batch)
    res = psi_xx + E * psi
    LPDE = tf.reduce_mean(tf.square(res))
    psi2 = tf.squeeze(tf.square(psi), axis=1)
    xb   = tf.squeeze(tf.convert_to_tensor(x_batch), axis=1)
    dx   = xb[1:] - xb[:-1]
    integral = tf.reduce_sum(0.5*(psi2[1:] + psi2[:-1]) * dx)
    Lnorm = tf.square(integral - 1.0)
    L = LPDE + lam * Lnorm
    return L, LPDE, Lnorm, integral

def run_one_mode_pinn(n, force=False):
    """
    Entrena (o carga de caché) la PINN para modo n, con E fija.
    Guarda/lee de BASE_DIR/pinn/n{n}.npz y figuras en BASE_DIR/pinn/.
    """
    cache_npz = path("pinn", f"n{n}.npz")
    fig_loss  = path("pinn", f"loss_n{n}.png")
    fig_modo  = path("pinn", f"modo_n{n}.png")

    # Si hay caché y no se fuerza, cargar y salir
    if (not force) and os.path.isfile(cache_npz):
        data = np.load(cache_npz, allow_pickle=True)
        return {
            "n": int(data["n"]),
            "xs": data["xs"],
            "psi_pred": data["psi_pred"],
            "psi_exact": data["psi_exact"],
            "E": float(data["E"]),
            "L2": float(data["L2"]),
            "integral": float(data["integral"]),
            "loss_path": fig_loss,
            "fig_path": fig_modo,
        }

    # Hiperparámetros heurísticos por n
    E = np.float32((n * math.pi)**2)
    USE_SINE = True if n >= 3 else False
    HIDDEN   = 128 if n >= 3 else 64
    N_col    = max(1024, 2048*n)
    EPOCHS   = 15000 if n >= 4 else (9000 if n==3 else (6000 if n==2 else 4000))
    LR0      = 3e-4  if n >= 4 else (5e-4 if n==3 else (7e-4 if n==2 else 1e-3))
    lam_hi, lam_lo = (300.0, 80.0) if n >= 3 else (40.0, 15.0 if n==2 else 10.0)

    net = make_net(n=n, hidden=HIDDEN, use_sine=USE_SINE)
    x_col = np.linspace(0,1,N_col, dtype=np.float32).reshape(-1,1)
    x_batch = tf.constant(x_col)

    lr_sched = keras.optimizers.schedules.PolynomialDecay(
        initial_learning_rate=LR0, decay_steps=EPOCHS,
        end_learning_rate=LR0*0.1, power=1.0
    )
    opt = keras.optimizers.Adam(learning_rate=lr_sched, clipnorm=1.0)

    loss_total, loss_pde, loss_norm = [], [], []
    for ep in range(1, EPOCHS+1):
        lam = lam_hi if ep < EPOCHS//3 else lam_lo
        with tf.GradientTape() as tape:
            L, LPDE, Lnorm, integral = compute_losses(net, x_batch, E, lam)
        grads = tape.gradient(L, net.trainable_variables)
        opt.apply_gradients(zip(grads, net.trainable_variables))
        loss_total.append(float(L)); loss_pde.append(float(LPDE)); loss_norm.append(float(Lnorm))
        if ep % max(800, EPOCHS//6) == 0 or ep == 1:
            tf.print(f"[PINN] n={n} ep={ep} | LPDE={LPDE:.3e} Lnorm={Lnorm:.3e} L={L:.3e} ∫|ψ|²≈{integral:.3f}")

    # Curva de pérdida
    plt.figure(figsize=(7,5))
    plt.semilogy(loss_total, label="Total")
    plt.semilogy(loss_pde,   label=r"$\mathcal{L}_{PDE}$")
    plt.semilogy(loss_norm,  label=r"$\mathcal{L}_{norm}$")
    plt.xlabel("Épocas"); plt.ylabel("Pérdida"); plt.title(f"PINN — Curva de pérdida (n={n})")
    plt.legend(); plt.tight_layout()
    plt.savefig(fig_loss, dpi=160); plt.close()

    # Evaluación fina
    xs = np.linspace(0,1,2000, dtype=np.float32).reshape(-1,1)
    psi_pred  = net(xs).numpy().squeeze().astype(np.float64)
    psi_exact = analytic_psi(n, xs.squeeze()).astype(np.float64)
    dot = float(np.dot(psi_pred, psi_exact))
    sgn = np.sign(dot);  sgn = 1.0 if sgn == 0 else sgn
    psi_pred *= sgn
    L2 = float(np.sqrt(np.mean((psi_pred-psi_exact)**2)))
    integ = trapz_np(psi_pred**2, x=xs.squeeze())

    # Figura PINN vs exacta
    plt.figure(figsize=(7.2,4.2))
    plt.plot(xs.squeeze(), psi_pred,  label=f"PINN ψ{n}")
    plt.plot(xs.squeeze(), psi_exact, "--", color="gray", label=f"Exacta ψ{n}")
    plt.title(f"PINN vs Exacta — n={n} | E={(n*math.pi)**2:.2f} | L2={L2:.2e}")
    plt.xlabel("x"); plt.ylabel("ψ"); plt.legend()
    plt.tight_layout(); plt.savefig(fig_modo, dpi=170); plt.close()

    # Guardar caché
    np.savez(cache_npz, n=n, xs=xs.squeeze().astype(float),
             psi_pred=psi_pred.astype(float), psi_exact=psi_exact.astype(float),
             E=float(E), L2=float(L2), integral=float(integ))

    return {
        "n": n, "xs": xs.squeeze(), "psi_pred": psi_pred, "psi_exact": psi_exact,
        "E": float(E), "L2": float(L2), "integral": float(integ),
        "loss_path": fig_loss, "fig_path": fig_modo
    }

# ============ COMPARACIÓN PINN vs DF ============
def compare_pinn_fd(n_list, N_fd=800, force_pinn=False):
    # DF una sola vez (malla fija para todos)
    xs_fd, psi_fd_list, E_fd = fd_solve(n_modes=max(n_list), N=N_fd)

    # Export DF
    for idx, n in enumerate(range(1, max(n_list)+1), start=1):
        np.savez(path("fd", f"n{n}.npz"),
                 xs=xs_fd.astype(float),
                 psi=psi_fd_list[idx-1].astype(float),
                 E=float(E_fd[idx-1]))
    # Tabla de resumen
    summary_rows = []

    # Por cada n: correr/cargar PINN y comparar
    for n in n_list:
        pinn = run_one_mode_pinn(n, force=force_pinn)

        # Interpolar DF a la malla fina de PINN (o al revés)
        from numpy import interp
        xs_p = pinn["xs"]
        psi_fd = interp(xs_p, xs_fd, psi_fd_list[n-1]).astype(float)
        psi_fd, _ = normalize_to_one(xs_p, psi_fd)

        psi_ex = analytic_psi(n, xs_p).astype(float)

        # Errores L2 absolutos (no relativos) sobre malla xs_p
        L2_pinn_exact = float(np.sqrt(np.mean((pinn["psi_pred"] - psi_ex)**2)))
        L2_fd_exact   = float(np.sqrt(np.mean((psi_fd - psi_ex)**2)))
        L2_pinn_fd    = float(np.sqrt(np.mean((pinn["psi_pred"] - psi_fd)**2)))

        # Normalizaciones
        norm_pinn = trapz_np(np.abs(pinn["psi_pred"])**2, xs_p)
        norm_fd   = trapz_np(np.abs(psi_fd)**2, xs_p)

        # Energies
        E_an = analytic_E(n)
        E_df = float(E_fd[n-1])

        summary_rows.append([
            n, E_an, E_df, L2_pinn_exact, L2_fd_exact, L2_pinn_fd, norm_pinn, norm_fd
        ])

        # Figura comparativa PINN vs DF vs Exacta
        plt.figure(figsize=(7.6,4.6))
        plt.plot(xs_p, psi_ex,  "--", color="gray", label="Exacta")
        plt.plot(xs_p, pinn["psi_pred"],  "-",  label="PINN")
        plt.plot(xs_p, psi_fd, ":",  label="DF (interp.)")
        plt.title(f"n={n} | E_an={(E_an):.2f} | E_df={(E_df):.2f}\n"
                  f"L2(PINN,Exact)={L2_pinn_exact:.2e} | L2(DF,Exact)={L2_fd_exact:.2e} | L2(PINN,DF)={L2_pinn_fd:.2e}")
        plt.xlabel("x"); plt.ylabel("ψ")
        plt.legend()
        plt.tight_layout()
        plt.savefig(path("comp", f"comp_n{n}.png"), dpi=180)
        plt.close()

    # Guardar CSV resumen
    csv_path = path("comp", "resumen_pinn_vs_df.csv")
    with open(csv_path, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["n", "E_analitica", "E_df",
                    "L2(PINN,Exacta)", "L2(DF,Exacta)", "L2(PINN,DF)",
                    "Norm(PINN)", "Norm(DF)"])
        for row in summary_rows:
            w.writerow(row)

    # Figura global de errores vs n
    ns = [r[0] for r in summary_rows]
    L2_pinn_exact = [r[3] for r in summary_rows]
    L2_fd_exact   = [r[4] for r in summary_rows]
    L2_pinn_fd    = [r[5] for r in summary_rows]

    plt.figure(figsize=(7.2,4.4))
    plt.plot(ns, L2_pinn_exact, "o-", label="L2(PINN,Exacta)")
    plt.plot(ns, L2_fd_exact,   "s--", label="L2(DF,Exacta)")
    plt.plot(ns, L2_pinn_fd,    "d-.", label="L2(PINN,DF)")
    plt.yscale("log"); plt.xlabel("Modo n"); plt.ylabel("Error L2 (log)")
    plt.title("Errores vs n — PINN vs DF vs Analítica")
    plt.tight_layout()
    plt.savefig(path("comp", "errores_globales.png"), dpi=180)
    plt.close()

    print(f"[OK] Guardado CSV: {csv_path}")
    print(f"[OK] Figuras por modo: {path('comp', is_dir=True)}")
    return summary_rows

# ============ EJECUCIÓN RÁPIDA ============
# Configura aquí cuántos modos deseas y si quieres forzar re-entrenamiento.
N_MAX = 6
FORCE_TRAIN = False    # True para reentrenar PINN ignorando caché
N_FD_GRID = 800        # puntos totales (incluye extremos) para DF

rows = compare_pinn_fd(list(range(1, N_MAX+1)), N_fd=N_FD_GRID, force_pinn=FORCE_TRAIN)

# Muestra breve del resumen
from pprint import pprint
pprint(rows[:3])
print("\nSalidas en:")
print("  - PINN:        ", path('pinn', is_dir=True))
print("  - DF:          ", path('fd', is_dir=True))
print("  - Comparación: ", path('comp', is_dir=True))


2025-10-08 12:51:11.776501: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-08 12:51:11.819516: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-08 12:51:12.816997: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Política actual: <DTypePolicy "mixed_float16">


I0000 00:00:1759945873.812006   16346 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1607 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:01:00.0, compute capability: 8.9
2025-10-08 12:51:14.562046: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


[PINN] n=1 ep=1 | LPDE=2.114e-02 Lnorm=9.998e-01 L=4.001e+01 ∫|ψ|²≈0.000
[PINN] n=1 ep=800 | LPDE=1.938e-03 Lnorm=5.631e-09 L=1.938e-03 ∫|ψ|²≈1.000
[PINN] n=1 ep=1600 | LPDE=1.295e-03 Lnorm=1.005e-08 L=1.295e-03 ∫|ψ|²≈1.000
[PINN] n=1 ep=2400 | LPDE=7.278e-04 Lnorm=5.807e-08 L=7.284e-04 ∫|ψ|²≈1.000
[PINN] n=1 ep=3200 | LPDE=3.642e-04 Lnorm=3.883e-08 L=3.646e-04 ∫|ψ|²≈1.000
[PINN] n=1 ep=4000 | LPDE=1.882e-04 Lnorm=1.720e-10 L=1.882e-04 ∫|ψ|²≈1.000
[PINN] n=2 ep=1 | LPDE=1.091e+00 Lnorm=9.970e-01 L=4.097e+01 ∫|ψ|²≈0.002
[PINN] n=2 ep=1000 | LPDE=5.681e-04 Lnorm=3.050e-08 L=5.693e-04 ∫|ψ|²≈1.000
[PINN] n=2 ep=2000 | LPDE=2.131e-04 Lnorm=9.956e-09 L=2.132e-04 ∫|ψ|²≈1.000
[PINN] n=2 ep=3000 | LPDE=1.447e-04 Lnorm=7.982e-10 L=1.447e-04 ∫|ψ|²≈1.000
[PINN] n=2 ep=4000 | LPDE=6.218e-05 Lnorm=1.541e-08 L=6.241e-05 ∫|ψ|²≈1.000
[PINN] n=2 ep=5000 | LPDE=3.533e-05 Lnorm=3.553e-11 L=3.533e-05 ∫|ψ|²≈1.000
[PINN] n=2 ep=6000 | LPDE=2.304e-05 Lnorm=3.624e-11 L=2.304e-05 ∫|ψ|²≈1.000
[PINN] n=3 ep=1 | L